In [ ]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from imblearn.over_sampling import RandomOverSampler 
from imblearn.combine import SMOTETomek, SMOTEENN
import joblib

import os

# Defining the path for the csv file
path = os.path.join("dataset.csv")

# Storing the dataframe in a variable named dataset
dataset = pd.read_csv(path)

# Dropping the unnecessary columns
dataset = dataset.drop('seqn', axis='columns')
dataset = dataset.drop('Marital', axis='columns')
print(dataset.shape)
print(len(dataset[dataset['MetabolicSyndrome'] == 0]))
print(dataset.head())

In [ ]:
!pip3 install torch

In [ ]:
sex_mapping = {'Male': 0, 'Female': 1}
race_mapping = {'White': 0, 'Asian': 1, 'Black': 2, 'MexAmerican': 3, 'Hispanic': 4, 'Other': 5}

dataset['Sex'] = dataset['Sex'].replace(sex_mapping)
dataset['Race'] = dataset['Race'].replace(race_mapping)

# This is the incorrect implementation
'''
dataset = dataset.fillna(2)
dataset = dataset.fillna(4)
dataset = dataset.fillna(5)
'''
# Fill NaN values in column with index 2
dataset.iloc[:, 2] = dataset.iloc[:, 2].fillna(dataset.iloc[:, 2].mean())

# Fill NaN values in column with index 4
dataset.iloc[:, 4] = dataset.iloc[:, 4].fillna(dataset.iloc[:, 4].mean())

# Fill NaN values in column with index 5
dataset.iloc[:, 5] = dataset.iloc[:, 5].fillna(dataset.iloc[:, 5].mean())
print(dataset.head())

In [3]:
outcome_0 = dataset[dataset['MetabolicSyndrome'] == 0]
outcome_1 = dataset[dataset['MetabolicSyndrome'] == 1]


test_size_each_class = 400
test_0 = outcome_0.sample(n=test_size_each_class, random_state=42)
test_1 = outcome_1.sample(n=test_size_each_class, random_state=42)
#print(test_1)
test_data = pd.concat([test_0, test_1])

# Remove the test set rows from the original dataset to create the training set
train_data = dataset.drop(test_data.index)


x_train = train_data.drop('MetabolicSyndrome', axis=1).values
y_train = train_data['MetabolicSyndrome'].values
x_test = test_data.drop('MetabolicSyndrome', axis=1).values
y_test = test_data['MetabolicSyndrome'].values
from imblearn.over_sampling import SMOTE, ADASYN
from ctgan import CTGAN
smote = SMOTE()

In [4]:
def generate_synthetic_samples(train_data, method='combined', weights=(0.5, 0.5)):
    """
    Generate synthetic samples using a weighted combination of ADASYN and CTGAN.
    """
    train_data = train_data.dropna(subset=['MetabolicSyndrome'])

    X = train_data.drop('MetabolicSyndrome', axis=1)
    y = train_data['MetabolicSyndrome']

    # Identify discrete (categorical) columns in your dataset
    discrete_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Generate synthetic samples using ADASYN
    adasyn = ADASYN(random_state=42)
    X_adasyn, y_adasyn = adasyn.fit_resample(X, y)
    adasyn_samples = pd.DataFrame(X_adasyn[len(X):], columns=X.columns)

    # Generate synthetic samples using CTGAN
    ctgan = CTGAN(epochs=100)
    ctgan.fit(X, discrete_columns)
    ctgan_samples = ctgan.sample(len(adasyn_samples))
    ctgan_samples.columns = X.columns  # Ensure columns match original data

    if method == 'adasyn':
        final_synthetic = adasyn_samples
    elif method == 'ctgan':
        final_synthetic = ctgan_samples
    else:  # Combined method
        n_adasyn = int(weights[0] * len(adasyn_samples))
        n_ctgan = len(adasyn_samples) - n_adasyn

        final_synthetic = pd.concat([
            adasyn_samples.sample(n=n_adasyn, random_state=42, replace=True),
            ctgan_samples.sample(n=n_ctgan, random_state=42, replace=True)
        ])

    final_synthetic['MetabolicSyndrome'] = 1  # Assuming synthetic samples are for the minority class
    balanced_data = pd.concat([train_data, final_synthetic])

    return balanced_data


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from ctgan import CTGAN
import os

# Load the dataset
path = os.path.join("dataset.csv")
dataset = pd.read_csv(path)

# Drop unnecessary columns
dataset = dataset.drop(['seqn', 'Marital'], axis='columns')

# Map categorical variables to numerical values
sex_mapping = {'Male': 0, 'Female': 1}
race_mapping = {'White': 0, 'Asian': 1, 'Black': 2, 'MexAmerican': 3, 'Hispanic': 4, 'Other': 5}
dataset['Sex'] = dataset['Sex'].replace(sex_mapping)
dataset['Race'] = dataset['Race'].replace(race_mapping)

# Fill NaN values with the mean of the respective columns
dataset.iloc[:, 2] = dataset.iloc[:, 2].fillna(dataset.iloc[:, 2].mean())
dataset.iloc[:, 4] = dataset.iloc[:, 4].fillna(dataset.iloc[:, 4].mean())
dataset.iloc[:, 5] = dataset.iloc[:, 5].fillna(dataset.iloc[:, 5].mean())

# Clean the dataset before splitting
dataset = dataset.dropna(subset=['MetabolicSyndrome'])

# Split the data into training and test sets
outcome_0 = dataset[dataset['MetabolicSyndrome'] == 0]
outcome_1 = dataset[dataset['MetabolicSyndrome'] == 1]
test_size_each_class = 400
test_0 = outcome_0.sample(n=test_size_each_class, random_state=42)
test_1 = outcome_1.sample(n=test_size_each_class, random_state=42)
test_data = pd.concat([test_0, test_1])
train_data = dataset.drop(test_data.index)

# Define a function to generate synthetic samples using ADASYN and CTGAN
def generate_synthetic_samples(train_data, method='combined', weights=(0.5, 0.5)):
    train_data = train_data.dropna(subset=['MetabolicSyndrome'])

    X = train_data.drop('MetabolicSyndrome', axis=1)
    y = train_data['MetabolicSyndrome']

    # Identify discrete (categorical) columns in your dataset
    discrete_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Generate synthetic samples using ADASYN
    adasyn = ADASYN(random_state=42)
    X_adasyn, y_adasyn = adasyn.fit_resample(X, y)
    adasyn_samples = pd.DataFrame(X_adasyn[len(X):], columns=X.columns)

    # Generate synthetic samples using CTGAN
    ctgan = CTGAN(epochs=300)
    ctgan.fit(X, discrete_columns)
    ctgan_samples = ctgan.sample(len(adasyn_samples))
    ctgan_samples.columns = X.columns  # Ensure columns match original data

    if method == 'adasyn':
        final_synthetic = adasyn_samples
    elif method == 'ctgan':
        final_synthetic = ctgan_samples
    else:  # Combined method
        n_adasyn = int(weights[0] * len(adasyn_samples))
        n_ctgan = len(adasyn_samples) - n_adasyn

        final_synthetic = pd.concat([
            adasyn_samples.sample(n=n_adasyn, random_state=42, replace=True),
            ctgan_samples.sample(n=n_ctgan, random_state=42, replace=True)
        ])

    final_synthetic['MetabolicSyndrome'] = 1  # Assuming synthetic samples are for the minority class
    balanced_data = pd.concat([train_data, final_synthetic])

    return balanced_data

# Function to train and evaluate model
def evaluate_model(train_data, test_data):
    X_train = train_data.drop('MetabolicSyndrome', axis=1).values
    y_train = train_data['MetabolicSyndrome'].values
    X_test = test_data.drop('MetabolicSyndrome', axis=1).values
    y_test = test_data['MetabolicSyndrome'].values
    
    model = XGBClassifier(random_state=42, n_estimators=100, learning_rate=0.3, max_depth=3)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred)
    }

# Define a function to evaluate multiple weight combinations and store results in a CSV file
def evaluate_and_store_results(train_data, test_data):
    results_list = []

    # Iterate over lambda values from 0 to 1 with a step of 0.05
    for lambda_value in np.arange(0, 1.05, 0.05):
        weights = (lambda_value, 1 - lambda_value)
        balanced_data_combined = generate_synthetic_samples(train_data, method='combined', weights=weights)

        # Ensure sample size does not exceed population
        if len(balanced_data_combined) > len(train_data):
            balanced_data_combined = balanced_data_combined.sample(n=len(train_data), random_state=42, replace=True)

        results = evaluate_model(balanced_data_combined, test_data)

        # Store the results along with the lambda value and weights
        results_list.append({
            'lambda': lambda_value,
            'weights': weights,
            'accuracy': results['accuracy'],
            'precision': results['precision'],
            'recall': results['recall'],
            'f1': results['f1']
        })
        print(f"Lambda: {lambda_value:.2f}, Weights: {weights}, Results: {results}")

    # Convert the results list to a DataFrame and save it as a CSV file
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('adasyn_ctgan_evaluation_results_300.csv', index=False)

# Evaluate and store the results in a CSV file
evaluate_and_store_results(train_data, test_data)

print("Evaluation results have been stored in evaluation_results.csv")

Lambda: 0.00, Weights: (0.0, 1.0), Results: {'accuracy': 0.82875, 'precision': 0.8856304985337243, 'recall': 0.755, 'f1': 0.815114709851552}
Lambda: 0.05, Weights: (0.05, 0.95), Results: {'accuracy': 0.835, 'precision': 0.8764044943820225, 'recall': 0.78, 'f1': 0.8253968253968254}
Lambda: 0.10, Weights: (0.1, 0.9), Results: {'accuracy': 0.84125, 'precision': 0.8845070422535212, 'recall': 0.785, 'f1': 0.8317880794701987}
Lambda: 0.15, Weights: (0.15000000000000002, 0.85), Results: {'accuracy': 0.84, 'precision': 0.9, 'recall': 0.765, 'f1': 0.827027027027027}
Lambda: 0.20, Weights: (0.2, 0.8), Results: {'accuracy': 0.85, 'precision': 0.8867403314917127, 'recall': 0.8025, 'f1': 0.8425196850393701}
Lambda: 0.25, Weights: (0.25, 0.75), Results: {'accuracy': 0.845, 'precision': 0.8791208791208791, 'recall': 0.8, 'f1': 0.837696335078534}
Lambda: 0.30, Weights: (0.30000000000000004, 0.7), Results: {'accuracy': 0.8475, 'precision': 0.8904494382022472, 'recall': 0.7925, 'f1': 0.8386243386243386}

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming train_data and test_data are pandas DataFrames
# Generate synthetic data using the custom function
# 0.14289794020845348, 0.8571020597915465
balanced_train_data = generate_synthetic_samples(train_data, method='combined', weights=(0.14289794020845348, 0.8571020597915465))

# Split features and labels
X_train = balanced_train_data.drop('MetabolicSyndrome', axis=1)
y_train = balanced_train_data['MetabolicSyndrome']

X_test = test_data.drop('MetabolicSyndrome', axis=1)
y_test = test_data['MetabolicSyndrome']

# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


In [ ]:
import joblib
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import beta

# Function to evaluate the model
def evaluate_model(train_data, test_data, weights):
    # Generate synthetic data with the current weight combination
    balanced_train_data = generate_synthetic_samples(train_data, method='combined', weights=weights)

    # Split features and labels
    X_train = balanced_train_data.drop('MetabolicSyndrome', axis=1)
    y_train = balanced_train_data['MetabolicSyndrome']
    X_test = test_data.drop('MetabolicSyndrome', axis=1)
    y_test = test_data['MetabolicSyndrome']

    # Create and train the XGBoost model
    xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)
    xgb_model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = xgb_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return accuracy, precision, recall, f1, xgb_model

# Function to find the best weight combination
def find_best_weight_combination_beta(train_data, test_data, num_samples=100, alpha=2, beta_param=2):
    best_accuracy = 0
    best_weights = (0, 0)
    best_results = {}
    best_model = None

    # Sample weights from a Beta distribution
    for _ in range(num_samples):
        # Randomly sample a weight for ADASYN and CTGAN from a Beta distribution
        weight_adasyn = beta.rvs(alpha, beta_param)
        weight_ctgan = 1 - weight_adasyn

        weights = (weight_adasyn, weight_ctgan)

        # Evaluate the model with the current weights
        accuracy, precision, recall, f1, xgb_model = evaluate_model(train_data, test_data, weights)

        # If this combination performs better, update best weights and results
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_weights = weights
            best_results = {
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1
            }
            best_model = xgb_model

    return best_weights, best_results, best_model

# Run the weight optimization
optimal_weights, optimal_results, best_model = find_best_weight_combination_beta(train_data, test_data)

# Save the best model
joblib.dump(best_model, 'best_xgboost_model.pkl')

# Display the best weights and corresponding evaluation metrics
print(f"\nOptimal Weight Combination (ADASYN, CTGAN): {optimal_weights}")
print(f"\nOptimal Combined Method Results:")
print(optimal_results)
